In [1]:
import requests
import streamlit as st
from streamlit_lottie import st_lottie

import torch
from transformers import pipeline

from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

C:\Users\c062387\PythonEnvs\uw\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Models

In [2]:
dolly = 'C:/Users/c062387/OneDrive - Thrivent Financial/Documents/Documents/UW Analytics Team/GitRepos/question-answer-app/dolly/'
xdoc = 'C:/Users/c062387/OneDrive - Thrivent Financial/Documents/Documents/UW Analytics Team/GitRepos/question-answer-app/xdoc/squad2/'

def get_models(dolly, xdoc):
    generate_text = pipeline(model=dolly, torch_dtype=torch.bfloat16,
                            trust_remote_code=True, device_map="auto", return_full_text=True)

    qa_model = pipeline('question-answering', model=xdoc, tokenizer=xdoc)

    return generate_text, qa_model

generate_text, qa_model = get_models(dolly, xdoc)

# Dolly Template

In [4]:
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

# Ask questions
def ask_xdoc(question, context):
    return qa_model({'question': question,'context': context}).get('answer')

def ask_dolly(question, context):
    return llm_context_chain.predict(instruction=question, context=context).lstrip()

In [23]:
context = """
Tobacco products, especially when smoked or used orally, have negative effects on human health, and concerns about these effects have existed for a long time. Research has focused primarily on cigarette smoking.[1][2]

Tobacco smoke contains more than 70 chemicals that cause cancer.[3] It also contains nicotine, which is a highly addictive psychoactive drug. When tobacco is smoked, nicotine causes physical and psychological dependency. Cigarettes sold in underdeveloped countries tend to have higher tar content, and are less likely to be filtered, potentially increasing vulnerability to tobacco smoking–related diseases in these regions.[4]

According to the World Health Organization (WHO), "[Smoking and oral] Tobacco use is the single greatest cause of preventable death globally."[5] As many as half of people who smoke tobacco or use the substance orally, die from complications related to such use.[3] The WHO estimates that each year in total about 6 million people die from tobacco-related causes (about 10% of all deaths) with 600,000 of these occurring in non-smokers due to second-hand smoke.[3][6] It is estimated to have caused 100 million deaths in the 20th century.[3] Similarly, the United States Centers for Disease Control and Prevention describes smoking tobacco and oral use of tobacco as "the single most important preventable risk to human health in developed countries and an important cause of premature death worldwide."[7] Currently, the number of premature deaths in the U.S. from tobacco use per year outnumber the number of workers employed in the tobacco industry by 4 to 1.[8] According to a 2014 review in The New England Journal of Medicine, tobacco smoking will kill about 1 billion people in the 21st century if current smoking patterns persist, half of them before the age of 70.[9]

China has the largest tobacco smoking population, followed by India. India has highest tobacco chewing population in the world. 154 people die every hour in India due to chewing and smoking tobacco.[10][11]
"""

print(context)


Tobacco products, especially when smoked or used orally, have negative effects on human health, and concerns about these effects have existed for a long time. Research has focused primarily on cigarette smoking.[1][2]

Tobacco smoke contains more than 70 chemicals that cause cancer.[3] It also contains nicotine, which is a highly addictive psychoactive drug. When tobacco is smoked, nicotine causes physical and psychological dependency. Cigarettes sold in underdeveloped countries tend to have higher tar content, and are less likely to be filtered, potentially increasing vulnerability to tobacco smoking–related diseases in these regions.[4]

According to the World Health Organization (WHO), "[Smoking and oral] Tobacco use is the single greatest cause of preventable death globally."[5] As many as half of people who smoke tobacco or use the substance orally, die from complications related to such use.[3] The WHO estimates that each year in total about 6 million people die from tobacco-rel

In [27]:
question = 'Which contry has the largert tobacco smoking population?'

print('XDoc Response')
print(ask_xdoc(question, context))
print('\nDolly Response')
print(ask_dolly(question, context))

XDoc Response
China

Dolly Response
China has the largest tobacco smoking population, followed by India. India has highest tobacco chewing population in the world. 154 people die every hour in India due to chewing and smoking tobacco.
